In [0]:
import nltk

# Tokenization

Es el proceso de separar una sentencia en unidades minimas llamadas tokens.
Un token es una unidad util para el procesamiento semantico, puede ser una palabra, sentencia o parrafo.

Se puede separar por espacios, puntuaciones.
En ingles es facil separar por espacios, en cambio en aleman no, porque existen palabras que se escriben sin separación una de la otra, al igual el japones, no tiene espacios.

In [0]:
text = "This is Andrew's text, isn't it?"

In [9]:
tokenizer = nltk.tokenize.WhitespaceTokenizer() # problema con is y isn't ambos el mismo significado
tokenizer.tokenize(text)

['This', 'is', "Andrew's", 'text,', "isn't", 'it?']

In [10]:
tokenizer = nltk.tokenize.WordPunctTokenizer() # problema con 's', 'isn', 't'
tokenizer.tokenize(text)

['This', 'is', 'Andrew', "'", 's', 'text', ',', 'isn', "'", 't', 'it', '?']

In [11]:
tokenizer = nltk.tokenize.TreebankWordTokenizer() # "'s" y "n't" tienen mas significado
tokenizer.tokenize(text)

['This', 'is', 'Andrew', "'s", 'text', ',', 'is', "n't", 'it', '?']

# Token Normalization
## Stemming 
Proceso de remover y reemplazar el sufijo, para obtener la raiz de la palabra, llamada "stem"

Ejemplo:

Porter's stemmer

Tiene 5 fases heuristicas de palabras reducidas y aplicadas secuencialmente.

Regla

> SSES -> SS       caresses -> caress

> IES -> I              ponies     -> poni

> SS -> SS           caress     -> caress

> S ->                   cats         -> cat

Sin embargo es un probelma para verbos irregulares y produce palabras sin significado, wolves -> wolv

## Lemmatization
Es la forma base de diccionario de una palabra, conocido como "lemma"

Ejemplo

WordNet Lemmatizer





In [13]:
text = "feet cats wolves talked"
tokenizer = nltk.tokenize.TreebankWordTokenizer()
tokens = tokenizer.tokenize(text)
tokens

['feet', 'cats', 'wolves', 'talked']

In [14]:
stemmer = nltk.stem.PorterStemmer()
" ".join(stemmer.stem(token) for token in tokens)

'feet cat wolv talk'

In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [17]:
stemmer = nltk.stem.WordNetLemmatizer()
" ".join(stemmer.lemmatize(token) for token in tokens)

'foot cat wolf talked'

Problemas

Uso de mayúsculas

Us, us, US

Soluciones:

> Lowercasing al comienzo de la oración
> Lowercasing palabras en los títulos
> Dejar las palabras que estan en el medio de la oracion tal cual.

Uo de acrónimos



# Transformar tokens en features
## BOW 
Cuenta el número de veces que se repite una palabra en el texto.
Cada token sera una columna en nuestro vector característico, este proceso se llama "text 
vectorization"
                          
                  good |movie|not|a|did|like
---


buena pelicula                      1 1 0 0 0 0


---


la pelicula no es buena       1 1 1 1 0 0


---


no me gusta                         0 0 1 0 1 1


---



Problemas:
> Se pierde el orden de las palabras
>> Contar pares, triples de palabras para conservar un orden local entre palabras, desventaja esta representacion aumenta el numero de features. Pero se puede remover algunos n-grams de acuerdo a la frequencia en la que aparecen en el corpus.
>>> Remover n-grams de frequencia alta

>>>> Articulos, preposiciones, los llamados stop-words que no ayudan a discriminar textos.

>>> Remover n-grams de baja frequencia

>>>> typos = palabras con error ortográfico o n-grams raros, removerlos porque podrían generar overfitting

>>> Quedarse con n-grams de frequencia media

>>>> Como saber cual es la mejor frequencia media

> El contador no esta normalizado


Filtrar los n-grams de frequencia media por medio de un ranking.
El n-gram con menor frequencia puede ser una palabra específica y útil para discriminar el texto ejempo Wi-fi breaks.

##N-GRAMS
###Métodos para rankear n-grams
#### Term Frequency TF

tf(t,d) frequencia de un ttermino en un documento

binary -> 1,0 # token presente o no en el documento

raw count -> f,t,d # distribucion de probabilidade de los tokens

term frequency -> $f_{t,d}/\sum_{t' \in d} f_{t',d}$ # distribución normalizada

log normalization -> $1 + log(f_{t,d})$ 

#### Inverse document frequency
Cuantos documentos contienen un especifico termino.

>$N = |D|$ -> número total de documentos en el corpus.
>|{d \in D:t \in d}| -> número de documentos donde el término $t$ aparece.

$$idf(t,D) = \log\frac{N}{|\{d \in D: t \in d\}|}$$

#### TF-IDF
$$tfidf(t,d,D) = tf(t,d) \dot idf(t,D)$$

Un valor alto de TF-IDF se da cuando el término tiene una frequencia alta en el propio texto y una frequencia baja entre los demas documentos.




In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
texts = ["good movie", "not a good movie", "did not like", "i like it", "good one"]
"""
min_df (ratio de documentos) parámetro para setear un umbral minimo de frequencias del documento, low frequency n-grams 
max_df (ratio de documentos) stop-words 
ngram_range rango de ngram-s de tamanho 1 y 2 por ejemplo.
"""
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))
features = tfidf.fit_transform(texts)
pd.DataFrame(features.todense(),columns=tfidf.get_feature_names())

,good movie,like,movie,not
0,0.707107,0.000000,0.707107,0.000000
1,0.577350,0.000000,0.577350,0.577350
2,0.000000,0.707107,0.000000,0.707107
3,0.000000,1.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000
